## Contexto

Una entidad educativa se encuentra en un proceso de migración de sistemas, cuyo objetivo busca lograr la limpieza y estandarización de sus datos.
Para ello provee fuentes de datos contenidos en diferentes archivos en formato csv.

Dicha entidad contrata nuestros servicios para crear una aplicación que permita la importación de los datos a una base de datos, con las siguientes consideraciones:

* Los datos insertados en las diferentes tablas deben ser únicos, por lo que se deberá revisar la consistencia y eliminar aquellos que estén repetidos. El campo de personal_id (DNI) es de tipo texto y hay que transformarlo en tipo número. (O)
* Cada tabla creada debe contar un un campo que haga referencia a la fecha y hora de la inserción del registro, como así también otro con la fecha y hora de modificación. (J)
* Se debe analizar toda la información de cada fuente de datos y diseñar un modelo de entidad / relación que permita disgregar en tablas principales y de catálogo manteniendo la respectiva relación.(ejemplo género, localidades, etc.) (J)
* Las tablas catálogo deben ser universales, de manera que puedan ser relacionadas por mas de una tabla actual o futura. (J)
* La solución debe ser modularizada, con el fin de poder reutilizar parte del código en futuros desarrollos sin que esto afecte su normal funcionamiento. (O)
* Cada función o procedimiento debe estar debidamente documentada para que cualquier equipo de desarrollo actual o futuro conozca el funcionamiento si necesidad de leer/interpretar el código. (A)
* Luego de la migración de los datos, se debe entregar reportes para respaldar la integridad de la información. (A)

## Metadata

#### Alumnos (alumnos.csv)
1. first_name: Nombre del alumno
2. last_name: Apellido del alumno
3. gender: Género
4. birthdate: Fecha de nacimiento
5. personal_id: Número de identificación del alumno (Cédula de identidad)
6. country: Pais
7. city: Ciudad
8. town: Municipio 
9. state: Provincia 
10. program: Carrera que cursa el alumno

#### Profesores (profesores.csv)
*Un profesor puede estar inscripto en mas de una carrera, pero no puede estar mas de una vez en la misma.*
1. first_name: Nombre del profesor
2. last_name: Apellido del profesor
3. gender: Género
4. birthdate: Fecha de nacimiento
5. personal_id: Número de identificación del profesor (Cédula de identidad)
6. country: Pais
7. city: Ciudad
8. town: Municipio 
9. state: Provincia 

### Distribución de carreras y profesores (cursos_profesores.csv)
1. program: Carrera
2. branch:Rama
3. institute: Facultad
4. campus: Nombre del campus
5. instructor: Identificación personal del profesor (Cédula de identidad)


## Reportes

La aplicación debe contar con la capacidad de generar al menos los siguientes reportes:  

* Listado con la cantidad de alumnos por carreas  
* Listado con la cantidad de profesores por carreas  
* Listado con el detalle de la cantidad de alumnos por provincia de origen y carrera que cursan  
* Distribución de alumnos por carrea y grupo etario (<20 años, entre 20 años y 30 años, >30 años)  
* Listado de profesores relacionado con cada alumno  


## Tener en cuenta

* Tablas de catálogo (no principales) a hacer:
    * -Una con el género
    * -Una con la localidad (Ciudad - Municipio - Provincia)
    * -Una con las carreras (Programa). A su vez esta tabla va a estar vinculada a la de instituciones, ramas y campus.
    * -Una con la institución
    * -Una con la rama
    * -Una con el campus

    Después los identificadores (primary key) de estas tablas hay que insertarlos como claves foráneas en las tablas de alumnos y profesores. Entre carreras y profesores (esta ya existe, es la de cursos_profesores) y entre carreras y alumnos se va a formar una tabla intermedia.

* El campo de personal_id (DNI) es de tipo texto y hay que transformarlo en tipo número. Para esto hay que recorrer la lista de diccionarios (el dataframe) con un for, y asignarle un nuevo valor a la clave personal_id que sería el valor original pero int y usando el método replace para reemplazar los puntos por "".

* Hay que verificar que los datos no se repitan. En primer lugar hay que comparar los registros enteros (el diccionario) para ver si hay alguno repetido, y en segundo lugar hay que controlar que los campos que son únicos, como personal_id, no se repitan.

    <!- para eliminar registros repetidos -->

    l = [{"a":123,"b":1234},{"a":3222,"b":1234},{"a":123,"b":1234},{"a":123,"b":3333}]

    lista_unica = set()    
    nueva_lista = []       

    for d in l:

        t = tuple(d.items())
        if t not in lista_unica:
        
            lista_unica.add(t)
            nueva_lista.append(d)

    print (lista_unica)   
    print (nueva_lista)

    <!- para eliminar valores repetidos -->
    
    l = [{"a":123,"b":1234},{"a":3222,"b":1234},{"a":123,"b":1234},{"a":123,"b":3333}]

    
    valores_unicos = [] 
    valores_repetidos = [] 

    for x in l:
        for i,j in x.items():      
        if i=="a":          
                if j not in valores_unicos:        
                    valores_unicos.append(j)              
                else:            
                    valores_repetidos.append (l[l.index(x)])        
    for x in l:
        if x in valores_repetidos:
            del l[l.index(x)]

    print(valores_repetidos)
    print(l)  


* Modularizar el código implica crear diferentes archivos python (módulos) que se importan después en el principal. Por ejemplo la configuración de la conexión a la base de datos se pone en un módulo (el host, el puerto, etc.) y las clases se ponen en otro módulo, que a su vez importa el de la conexión. Después el principal importa el de las clases y a través de este importa el de la conexión.
* Campo de fecha y hora del registro: 

    insert_date = DateTimeField (default=datetime.datetime.now())

   Campo de actualización de fecha y hora del registro: 

    update_date = DateTimeField()

  Campo de estado del registro (A=activo, B=baja), se usa para no tener que borrar el registro:
  
    status= CharField(default="A")